<B>Setup your dataframe using SQL</B>

In [0]:
df = sqlContext.sql("SELECT \
  a.Name \
, a.Platform as Console \
, CASE \
    WHEN a.Platform IN ('PS3','PS','PS2','PS4','PSP','PSV') THEN 'Sony' \
    WHEN a.Platform IN ('3DS','WiiU','GBA','GC','Wii','DS') THEN 'Nintendo' \
    WHEN a.Platform IN ('XOne','X360','XB') THEN 'Microsoft' \
  END Console_Company \
, CAST(a.Year_of_Release AS INT) Year_of_Release \
, a.Genre \
, CASE \
    WHEN a.Publisher LIKE 'Sony%' THEN 1 \
    WHEN a.Publisher IN ('Microsoft','Nintendo') THEN 1 \
    ELSE 0 \
  END AS is_Console_Publisher_Match \
, a.Global_Sales \
, a.Critic_Score \
, a.Critic_Count \
, CAST(a.User_Score * 10 AS INT) User_Score \
, a.User_Count \
, CASE \
    WHEN a.Rating IN ('E','EC','K-A') THEN 7 \
    WHEN a.Rating = 'E10+' THEN 10 \
    WHEN a.Rating = 'T' THEN 13 \
    WHEN a.Rating = 'M' THEN 17 \
    WHEN a.Rating = 'AO' THEN 18 \
  END Rating_Age \
, CASE \
    WHEN a.Platform IN ('3DS','PSP','GBA','PSV','DS') THEN 1 ELSE 0 END AS is_Handheld \
, CASE WHEN c.Name IS NULL THEN 0 ELSE 1 END AS is_Exclusive \
, CASE WHEN a.NAME LIKE '%Mario%' THEN 1 ELSE 0 END AS is_Mario \
FROM \
  video_games_sales a LEFT JOIN \
  (\
    SELECT \
    Name \
  , Year_of_Release \
  FROM \
    video_games_sales \
  GROUP BY \
    Name \
  , Year_of_Release \
  HAVING \
    COUNT(DISTINCT platform) = 1 \
  ) c ON a.Name = c.Name AND a.Year_of_Release = c.Year_of_Release \
WHERE \
  a.Platform NOT IN ('PC','DC') \
  AND a.User_Score NOT IN ('','tbd')\
  AND a.Critic_Score IS NOT NULL\
  AND a.User_Score IS NOT NULL\
  AND a.Publisher IS NOT NULL\
  AND a.Developer IS NOT NULL\
  AND a.Rating <> 'RP' \
  AND a.Year_of_Release IS NOT NULL \
  AND a.Year_of_Release NOT IN ('N/A') \
  AND a.Name NOT IN ('Wii Sports Resort','Wii Sports','Wii Play','Wii Fit Plus','Wii Fit','Kinect Adventures!') \
  ")
                    

In [0]:
df.printSchema()

root
-- Name: string (nullable = true)
-- Console: string (nullable = true)
-- Console_Company: string (nullable = true)
-- Year_of_Release: integer (nullable = true)
-- Genre: string (nullable = true)
-- is_Console_Publisher_Match: integer (nullable = false)
-- Global_Sales: double (nullable = true)
-- Critic_Score: integer (nullable = true)
-- Critic_Count: integer (nullable = true)
-- User_Score: integer (nullable = true)
-- User_Count: integer (nullable = true)
-- Rating_Age: integer (nullable = true)
-- is_Handheld: integer (nullable = false)
-- is_Exclusive: integer (nullable = false)
-- is_Mario: integer (nullable = false)

In [0]:
df.groupBy('Rating_Age').count().show()

+----------+-----+
Rating_Age|count|
+----------+-----+
 13| 1720|
 17| 1008|
 7| 1657|
 10| 718|
 18| 1|
+----------+-----+

In [0]:
df.show()

+--------------------+-------+---------------+---------------+-----+--------------------------+------------+------------+------------+----------+----------+----------+-----------+------------+--------+
 Name|Console|Console_Company|Year_of_Release|Genre|is_Console_Publisher_Match|Global_Sales|Critic_Score|Critic_Count|User_Score|User_Count|Rating_Age|is_Handheld|is_Exclusive|is_Mario|
+--------------------+-------+---------------+---------------+-----+--------------------------+------------+------------+------------+----------+----------+----------+-----------+------------+--------+
Universal Studios...| GC| Nintendo| 2001| Misc| 0| 0.1| 39| 11| 29| 11| 13| 0| 1| 0|
Rolling Stone: Dr...| Wii| Nintendo| 2009| Misc| 0| 0.1| 32| 4| 18| 4| 13| 0| 1| 0|
Pinball Hall of F...| 3DS| Nintendo| 2011| Misc| 0| 0.1| 67| 5| 73| 8| 10| 1| 1| 0|
 Princess Debut| DS| Nintendo| 2008| Misc| 0| 0.1| 71| 6| 74| 7| 7| 1| 1| 0|
 Patapon 2| PSP| Sony| 2008| Misc| 1| 0.1| 81| 49| 83| 62| 7| 1| 1| 0|
Family Feud: 2012...| X360| Microsoft| 2011| Misc| 0| 0.1| 47| 4| 53| 6| 10| 0| 0| 0|
 DJ Max Fever| PSP| Sony| 2009| Misc| 0| 0.11| 77| 5| 65| 13| 13| 1| 1| 0|
Karaoke Revolutio...| X360| Microsoft| 2008| Misc| 0| 0.11| 70| 7| 70| 6| 7| 0| 0| 0|
 Odama| GC| Nintendo| 2006| Misc| 1| 0.11| 62| 42| 80| 17| 10| 0| 1| 0|
 Truth or Lies| X360| Microsoft| 2010| Misc| 0| 0.11| 28| 6| 52| 6| 13| 0| 0| 0|
 Zubo| DS| Nintendo| 2008| Misc| 0| 0.11| 75| 19| 76| 75| 10| 1| 1| 0|
 Ping Pals| DS| Nintendo| 2004| Misc| 0| 0.11| 28| 17| 39| 26| 7| 1| 1| 0|
Mega Man Battle C...| GBA| Nintendo| 2003| Misc| 0| 0.11| 54| 25| 83| 8| 7| 1| 1| 0|
Marvel Trading Ca...| DS| Nintendo| 2007| Misc| 0| 0.12| 61| 17| 49| 7| 13| 1| 1| 0|
Power Gig: Rise o...| X360| Microsoft| 2010| Misc| 0| 0.12| 36| 20| 21| 17| 13| 0| 0| 0|
Pinball Hall of F...| X360| Microsoft| 2009| Misc| 0| 0.12| 83| 18| 83| 16| 10| 0| 0| 0|
Hatsune Miku: Pro...| PSV| Sony| 2016| Misc| 0| 0.12| 83| 5| 80| 22| 13| 1| 0| 0|
 Everybody Dance| PS3| Sony| 2011| Misc| 1| 0.12| 66| 5| 68| 8| 13| 0| 1| 0|
Chessmaster: The ...| PSP| Sony| 2008| Misc| 0| 0.12| 64| 4| 84| 18| 7| 1| 1| 0|
Mega Man X Collec...| GC| Nintendo| 2006| Misc| 0| 0.13| 75| 23| 92| 12| 7| 0| 1| 0|
+--------------------+-------+---------------+---------------+-----+--------------------------+------------+------------+------------+----------+----------+----------+-----------+------------+--------+
only showing top 20 rows

<B>Convert Catagorical Values into Numeric Values</B>

Use one-hot encoding (StringIndexer,OneHotEncoder)

In [0]:
from pyspark.ml.feature import OneHotEncoder, StringIndexer

In [0]:
categoricalCols = ['Console_Company','Console','Genre']
numericCols = ['Year_of_Release','Critic_Score','Critic_Count','User_Score','User_Count','Rating_Age','is_Exclusive', 'is_Console_Publisher_Match','is_Handheld','is_Mario']

In [0]:
indexOutputCols = [x + "Index" for x in categoricalCols]
oheOutputCols = [x + "OHE" for x in categoricalCols]

In [0]:
stringIndexer = StringIndexer(inputCols=categoricalCols
                             ,outputCols=indexOutputCols)
oheEncoder = OneHotEncoder(inputCols=indexOutputCols
                           ,outputCols=oheOutputCols)

In [0]:
indexed = stringIndexer.fit(df).transform(df)
encoded = oheEncoder.fit(indexed).transform(indexed)

In [0]:
encoded.printSchema()

root
-- Name: string (nullable = true)
-- Console: string (nullable = true)
-- Console_Company: string (nullable = true)
-- Year_of_Release: integer (nullable = true)
-- Genre: string (nullable = true)
-- is_Console_Publisher_Match: integer (nullable = false)
-- Global_Sales: double (nullable = true)
-- Critic_Score: integer (nullable = true)
-- Critic_Count: integer (nullable = true)
-- User_Score: integer (nullable = true)
-- User_Count: integer (nullable = true)
-- Rating_Age: integer (nullable = true)
-- is_Handheld: integer (nullable = false)
-- is_Exclusive: integer (nullable = false)
-- is_Mario: integer (nullable = false)
-- Console_CompanyIndex: double (nullable = false)
-- ConsoleIndex: double (nullable = false)
-- GenreIndex: double (nullable = false)
-- Console_CompanyOHE: vector (nullable = true)
-- ConsoleOHE: vector (nullable = true)
-- GenreOHE: vector (nullable = true)

<B>Create Feature column</B>

In [0]:
from pyspark.ml.linalg import Vectors
from pyspark.ml.feature import VectorAssembler

In [0]:
train_data,test_data = encoded.randomSplit([0.7,0.3])

In [0]:
assemblerInputs = oheOutputCols + numericCols

In [0]:
assembler = VectorAssembler(inputCols=assemblerInputs,outputCol="features")

In [0]:
output = assembler.transform(train_data)

<B> Linear Regression </B>

In [0]:
from pyspark.ml.regression import LinearRegression

In [0]:
lr = LinearRegression(featuresCol='features',labelCol="Global_Sales",maxIter=20)
lrModel = lr.fit(output)

In [0]:
type(lrModel)

Out[144]: pyspark.ml.regression.LinearRegressionModel

<B>Create a Pipeline</B>

Apply the Model to the Test Data Set

In [0]:
from pyspark.ml import Pipeline

In [0]:
pipeline = Pipeline(stages=[assembler, lr])
pipelineModel = pipeline.fit(test_data)

In [0]:
test_df = pipelineModel.transform(test_data)

Review Actual North American Sales vs Prediction

In [0]:
test_df.select('Name','Console','Year_of_Release','Global_Sales','prediction').sort(test_df.prediction.desc()).show()

+--------------------+-------+---------------+------------+------------------+
 Name|Console|Year_of_Release|Global_Sales| prediction|
+--------------------+-------+---------------+------------+------------------+
Uncharted 4: A Th...| PS4| 2016| 5.38|13.679215009850878|
 Grand Theft Auto V| PS3| 2013| 21.04| 8.574548183687952|
 Fallout 4| PS4| 2015| 7.16| 7.769804775981413|
The Elder Scrolls...| X360| 2011| 8.79| 7.757843575605833|
Metal Gear Solid ...| PS4| 2015| 3.41| 7.684672131323055|
 Mass Effect 2| X360| 2010| 3.1| 7.01153379085774|
Call of Duty: Mod...| X360| 2009| 13.47| 6.727717276258346|
 Heavy Rain| PS3| 2010| 3.03| 6.033982234899227|
 Mario Kart 8| WiiU| 2014| 7.09| 5.769543864112691|
inFAMOUS: Second Son| PS4| 2014| 2.79| 5.697527049383908|
 Red Dead Redemption| X360| 2010| 6.32| 5.695099086343504|
 Final Fantasy XIII| PS3| 2009| 5.33| 5.639863847304014|
 Titanfall| XOne| 2014| 2.95| 5.622902727420978|
Call of Duty: Mod...| PS3| 2009| 10.6| 5.509272267057256|
Batman: Arkham Kn...| PS4| 2015| 3.95| 5.26341581385617|
 Batman: Arkham City| PS3| 2011| 5.48| 5.239347687563026|
The Legend of Zel...| Wii| 2011| 3.95| 5.194364132783065|
 Beyond: Two Souls| PS3| 2013| 1.63| 5.081044970991812|
 Fallout 3| X360| 2008| 4.93| 4.265252724857447|
 Battlefield 3| PS3| 2011| 7.17| 4.260699142376012|
+--------------------+-------+---------------+------------+------------------+
only showing top 20 rows

Evaluate Model

In [0]:
sales_prediction = lrModel.evaluate(test_df.select('features','Global_Sales'))

In [0]:
sales_prediction.rootMeanSquaredError

Out[150]: 1.2073985950044996

In [0]:
test_data.select('Global_Sales').describe().show()

+-------+------------------+
summary| Global_Sales|
+-------+------------------+
 count| 1541|
 mean|0.9222193380921496|
 stddev| 1.500570499705639|
 min| 0.1|
 max| 21.04|
+-------+------------------+

In [0]:
sales_prediction.r2

Out[152]: 0.3521559988057338